In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from astroML.datasets import fetch_sdss_corrected_spectra

# Classification: part 1

The goal of a classification problem is to train a model that will successfully predict the classes of new objects that we feed in to the model. We do this a lot in astronomy! 

* Type 1 / Type 2 anything
* RR lyrae: RRab / RRc / RRd
* Seyfert 1 / 2 galaxies
* Mini-neptunes / Super Earths / etc.
* Blue cloud / red sequence / green valley galaxies
* ...the list goes on

Sometimes our classes are observationally determined, in which case we then try to make causal relationships to theory. Other times theory predicts clear separation of objects. In either case, we often want to be able to classify new objects, or select objects of a particular class. Sometimes it makes sense to draw a hard boundary and do hard classification, other times we prefer to have probabilities of belonging to classes, i.e. soft classification. There are reasons to do both in science, and there are many well-studied and used algorithms for classification problems.

Deciding on a classifier or classification method is generally very context dependent, and depends on whether you want soft/hard classifications, have many or few classes, have many or few data points, have a training set or not. Even within a given model, we often then have to choose a way to "score" the performance of the model. As with other machine learning methods, you also have to decide on what features to compute or use from the data or things you want to classify.

In choosing a classifier, it's important to weight several things:
* Do I care about population statistics? Does this classifier create a really complex selection function?
* How fast do I need the prediction to be?
* How much data do I have?
* How many features do I have? (i.e. dimensionality)

There are

To name a few classifiers:
* Nearest neighbor (discriminative):
    * +Very simple
    * +Can compare well to very complicated methods if you have a lot of data
    * +Non-parametric representation of complex decision boundaries "for free"
    * -Doesn't scale well to high dimensions
    * -Have to weight features or normalize in some way to compute distance
    * -Slow prediction (no real training)
* (Naive) Bayes (generative):
    * +Approximation to Bayesian hierarchical inference
    * +Simple, and therefore fast
    * +Interpretable, especially if your density distributions are probability distributions
    * +Can handle missing data
    * -Naive Bayes can't handle correlated features (generic Bayes is fine though)
* Decision trees / random forest (discriminative):
    * +Non-parametric
    * +Handles high-dimensional feature spaces
    * +Will tell you what features are most discriminating
    * -Have to rebuild full tree any time you get new data
    * -Susceptible to over-fitting
* Neural networks (discriminative or generative):
    * +Non-parametric
    * +Can extend to handle convolutional relationships between features
    * +All the rage right now, so good tools exist
    * -Designing good network architectures is an art, often done by intuition, not theory
    * -Generally require a lot of data and time to train
    * -Not interpretable
    
This week, we'll discuss nearest neighbors and simple decision trees.

## Data munging

In [ ]:
data = fetch_sdss_corrected_spectra()

In [ ]:
data.keys()

We need do some pre-processing to condense our labels a bit. In the original data, the value of `lineindex_cln` is:
* 4 if HII region
* 5 if AGN
* 0, 1, 2, 3, 6, 9 else (other things)

We'll make a new label array, `y`, that is:
* 0 if else
* 1 if HII
* 2 if AGN

These classes were determined using the line ratios, but we're going to use the spectral fluxes themselves as features to try to predict these classes.

In [ ]:
nspectra = len(data['lineindex_cln'])
y = np.zeros(nspectra)
y[data['lineindex_cln'] == 4] = 1
y[data['lineindex_cln'] == 5] = 2

Here's what the data look like in the line ratio space:

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
cs = ax.scatter(data['log_NII_Ha'], data['log_OIII_Hb'], 
                c=data['z'], vmin=0, vmax=0.2,
                marker='o', s=20, linewidth=0, alpha=0.4)
ax.set_xlim(-1.5, 0.5)
ax.set_ylim(-1.5, 1.5)
cb = fig.colorbar(cs)
cb.set_label('redshift, $z$')
ax.set_xlabel(r'$\log \left( [{\rm NII}] / {\rm H}\alpha \right)$')
ax.set_ylabel(r'$\log \left( [{\rm OIII}] / {\rm H}\beta \right)$')

We're going to prepare two feature sets that we'll use in various applications: 
1. A small section of the spectrum (~40 pixels or features)
2. The full spectra themselves (they have been put onto the same wavelength grid)

Let's look at a few of the spectra:

In [ ]:
rnd = np.random.RandomState(seed=42)

fig, axes = plt.subplots(4, 1, figsize=(12, 10), sharex=True, sharey=True)

for ax, i in zip(axes, rnd.choice(nspectra, size=len(axes), replace=False)):
    ax.plot(data['spectra'][i], marker='', drawstyle='steps-mid')
    ax.set_ylim(-5, 25)

And let's pack the spectra into feature matrices - in fact, Jake VdP has done that for us already! The full spectra have 1000 pixels (wavelength values), and our mini spectra will be a 40 pixel subset of the full spectra:

In [ ]:
X_spec_mini = data['spectra'][:, 200:240]
X_spec = data['spectra']
X_spec_mini.shape, X_spec.shape

The spectra have already been cleaned of any NaN or Inf values (set to 0).

Let's now take a look at a few applications.

---

# K Nearest Neighbors

We'll see how well we can do at predicting whether an object is an AGN vs. an HII region with KNN, after optimizing over the number of neighbors, `K`. For this example, we'll just use the line ratio data.

Convention is to define the classifier object as "`clf`". For the simplest version of this algorithm, it has 1 tunable parameter: the number of neighbors K to consider. 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
clf = KNeighborsClassifier(n_neighbors=5)

Let's start by fitting the classifier to all but the last 128 objects, then predict the classes of the last data points. In the case of KNN, what exactly is KNN doing when we "fit" the model? There are no parameters to fit for! Here it's just constructing a tree based on the features to help make the prediction step more efficient.

In [ ]:
clf.fit(X_spec_mini[:-128], y[:-128])

In [ ]:
y_pred = clf.predict(X_spec_mini[-128:])
y_true = y[-128:]

In [ ]:
print("Predicted:", y_pred[:16])
print("True:     ", y_true[:16])

Nice, our predictions are pretty good! But how do we evaluate how well our classifier is doing? We need to compute some kind of "score" for our classifier. In machine learning, this is often the most subtle challenge in the methodology. Be aware that scikit-learn often lets you specify or define your own score functions, but for now we're going to table this idea and stick to some simple defaults.

For discrete classification, a common score function is the "misclassification error." This is just the fractional number of data points that we incorrectly classified with whatever classification method we used. We can compute this fairly simply, and then take the compliment of the error to compute the "accuracy" of the classifier:

In [ ]:
misclassification_error = np.sum(y_true != y_pred) / len(y_true)
accuracy = 1 - misclassification_error
accuracy

That's a pretty good accuracy!

Scikit-learn also provides a function to compute this:

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_true, y_pred)

### Exercise: 

How does the accuracy change as you increase the number of neighbors ("K") from 1-10?

In [ ]:
Ns = np.arange(1, 15+1)
accs = []
for N in Ns:
    clf = KNeighborsClassifier(n_neighbors=N)
    clf.fit(X_spec_mini[:-1000], y[:-1000])
    
    y_pred = clf.predict(X_spec_mini[-1000:])
    y_true = y[-1000:]
    acc = accuracy_score(y_true, y_pred)
    accs.append(acc)
    
    print("n={0}, accuracy={1:.2f}".format(N, acc))

In [ ]:
plt.plot(Ns, accs, drawstyle='steps-mid', marker='')
plt.xlabel('$K$ neighbors')
plt.ylabel('Accuracy')
plt.xlim(0, 15.5)

---

## Cross-validation

Cross-validation provides a way to tune or set hyperparameters of machine learning algorithms to try to avoid overfitting to training data. The way these methods generally work for supervised learning problems (like classification) is by training the model on some subset of the full training set, predicting the labels of the held-back data, computing an accuracy metric by comparing to the true labels, then optimizing over the accuracy metric. With jargon, the labeled dataset is typically split into (at least two) subsets called the _training data_ and _test data_. The models are trained on the training data, and then evaluated on the test data, which the model has never seen before. Provided you separate the train/test data in a sensible way, this gives you an unbiased way to validate the models. 

Let's now use Sciki-learn's cross-validation score function to automatically do the train-test splitting (multiple times). How does the cross-validation score depend on `k`, the number of neighbors?

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
vals = np.arange(2, 100, 5)
mean_scores = []
for n_neighbors in vals:
    clf = KNeighborsClassifier(n_neighbors=n_neighbors)
    scores = cross_val_score(clf, X_spec_mini, y)
    mean_scores.append(np.mean(scores))

In [ ]:
plt.plot(vals, mean_scores)
plt.xlabel('$K$ neighbors')
plt.ylabel('Cross-validation accuracy')

**Question**: Why does the accuracy decrease when considering more neighbors? What is your expectation for the asymptotic behavior of this accuracy?

Scikit-learn also has a utility that will automatically do the optimization to find the model parameter that produces the highest-accuracy predictions (see [the documentation for some tips](http://scikit-learn.org/stable/modules/grid_search.html#grid-search-tips)):

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
knn_clf = KNeighborsClassifier()

__Exercise__: Split the above datasets into train and test sets. We'll need to split both the features (`X`) and the labels (`y`) in the same way:

In [ ]:
idx = np.random.choice(len(X_spec_mini), size=len(X_spec_mini), replace=False)

# 85% training data, 15% test data
split = int(0.85 * len(X_spec_mini))

train_X = X_spec_mini[idx[:split]]
train_y = y[idx[:split]]

test_X = X_spec_mini[idx[split:]]
test_y = y[idx[split:]]

Now use GridSearchCV to find the optimal number of neighbors

In [ ]:
params = {'n_neighbors': np.arange(2, 30, 1)}
cv_clf = GridSearchCV(knn_clf, param_grid=params)
cv_clf.fit(X_spec_mini, y)

In [ ]:
cv_clf.best_score_

In [ ]:
cv_clf.best_params_

---

# Naive Bayes

With Gaussians!

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
clf = GaussianNB()

In [ ]:
clf.fit(X_spec_mini, y)

In [ ]:
y_pred = clf.predict(X_spec_mini)
accuracy_score(y, y_pred)

**Question**: Why does Gaussian Naive Bayes perform so much worse than nearest neighbors?

A demonstration with a pathological data set:

In [ ]:
x1 = np.random.multivariate_normal([1, 1], np.array([[0.5, 0.499], [0.499, 0.5]])**2, size=1000)
x2 = np.random.multivariate_normal([1, 1], np.array([[0.5**2, -0.499**2], [-0.499**2, 0.5**2]]), size=1000)
X_patho = np.vstack((x1, x2))
y_patho = np.concatenate((np.zeros(len(x1)), np.ones(len(x2))))

# shuffle
idx = np.random.choice(X_patho.shape[0], size=X_patho.shape[0], replace=False)
X_patho = X_patho[idx]
y_patho = y_patho[idx]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.scatter(x1[:, 0], x1[:, 1])
ax.scatter(x2[:, 0], x2[:, 1])
ax.set_xlim(-1, 3)
ax.set_ylim(-1, 3)

In [ ]:
gnb = GaussianNB()
gnb.fit(X_patho[:-128], y_patho[:-128])

In [ ]:
y_patho_pred = gnb.predict(X_patho[-128:])
accuracy_score(y_patho[-128:], y_patho_pred)

---

# Decision trees

Make binary decisions for each feature with decision boundaries perpendicular to feature axes. Need to specify the depth/height of the tree, $D$ - this partitions the feature space into $2^D$ regions. Can set $D$ with cross-validation!

Can improve bias and variance using boosting, bagging.

For now, we'll just use a vanilla decision tree, but using the full spectrum as our feature set.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
clf = DecisionTreeClassifier(max_depth=8, 
                             min_samples_split=2, 
                             min_samples_leaf=1)

In [ ]:
clf.fit(X_spec[:-1024], y[:-1024])

In [ ]:
y_pred = clf.predict(X_spec[-1024:])
accuracy_score(y[-1024:], y_pred[-1024:])

**Exercise**: Use cross-validation to find the optimal `max_depth`. How does this compare to nearest neighbors?